In [22]:
import socket
import threading
from datetime import datetime
import os
import subprocess


In [2]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
#소켓이 꼭 TCPIP만 의미하는 것은 아니고 여러가지 통신방법이 있다.
#파라미터가 두 개 - 첫번째는 IP를 쓰겠다
#두번째는 TCP/UCP인데 STREAM은 TCP방법을 쓰겠다

server_socket.bind(('localhost', 12345))    #IP, 포트번호
server_socket.listen(0)                     #포트번호를 listening. 동시에 소켓을 몇개까지 연결할 건지. 0은 automatically
#대기모드
print('listening...')

client_socket, addr = server_socket.accept() #클라이언트 접속될 때까지는 대기상태
print('accepting')
data =client_socket.recv(65535) #클라이언트 접속이 되면 데이터를 읽어들임.
#데이터는(패킷은) 최대 64k. 더 작을수도 클 수도 있는데, 크면 쪼개서 전송된다.

print('receive >> ' + data.decode())  #unicode to 한글

client_socket.send(data)
print('send data')
client_socket.close()
print('종료')

listening...
accepting
receive >> hello
send data
종료


In [6]:
#simple http server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening...')
client_socket, addr = server_socket.accept()
print('accepting')
data =client_socket.recv(65535)

print('receive >> ' + data.decode())
client_socket.close()

listening...
accepting
receive >> POST / HTTP/1.1
Host: 127.0.0.1
Connection: keep-alive
Content-Length: 8
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
Origin: null
Content-Type: application/x-www-form-urlencoded
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36
Sec-Fetch-Dest: document
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Sec-Fetch-Site: cross-site
Sec-Fetch-Mode: navigate
Sec-Fetch-User: ?1
Accept-Encoding: gzip, deflate, br
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7

id=hello


In [7]:
#html file load
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening...')

if True :           #while True : 테스트 할 때는 1회접속이 편리
    client_socket, addr = server_socket.accept()
    print('accepting')
    data =client_socket.recv(65535)
    data = data.decode()

    headers = data.split("\r\n")
    filename = headers[0].split(" ")[1]
    
    header = 'HTTP/1.0 200 0K\r\n\r\n'
    file = open('.'+filename, 'rt', encoding='utf-8')
    html = file.read()
    #위 두 줄을 html = open('.'+filename, 'rt', encoding='utf-8').read() 로 하면 안 되더라

    client_socket.send((header+html).encode('utf-8'))
    client_socket.close()    #HTTP의 특성. 접속하고 끊어버린다.

listening...
accepting


In [17]:
#file 다루기
filename = '/index.html'

text = open('.'+filename, 'rt', encoding='utf-8').read()
#2nd param: read, write, binary, textflie
print(text)

<font color=red>Hello</font>


In [25]:
#확장자 나누기 + threading
def httpprocess(client_socket) :
    data =client_socket.recv(65535)
    data = data.decode()
    print(data)

    try:    
        headers = data.split("\r\n")
        filename = headers[0].split(" ")[1]
        _, ext = os.path.splitext(filename)

        group_a = ['.html', '.htm']
        group_b = ['.jpg', '.jpeg', '.png', '.bmp']

        
        if '.py' in filename:
            html = subprocess.check_output(['python.exe', '.'+filename])
            html = html.decode('cp949')
            header = 'HTTP/1.0 200 0K\r\n\r\n'
            client_socket.send((header+html).encode('utf-8'))
        elif ext in group_a :
            file = open('.'+filename, 'rt', encoding='utf-8')
            html = file.read()
            header = 'HTTP/1.0 200 0K\r\n\r\n'
            client_socket.send((header+html).encode('utf-8'))
        elif ext in group_b or ext == '.ico' :
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/png\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open('.' + filename, 'rb')
            client_socket.send(file.read())  #binary type
            file.close()
        else :
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'
            client_socket.send((header+html).encode('utf-8'))
    except Exception as e:
        print(e)
    client_socket.close()



server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind(('localhost', 80))
server_socket.listen(0)
print('listening...')

if True :
    client_socket, addr = server_socket.accept()
    client_socket.settimeout(3)
    print('accepting')
    t = threading.Thread(target=httpprocess, args=(client_socket,))
    t.start()

listening...
accepting
GET /test2.py HTTP/1.1
Host: localhost
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36
Sec-Fetch-Dest: document
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Sec-Fetch-Site: none
Sec-Fetch-Mode: navigate
Sec-Fetch-User: ?1
Accept-Encoding: gzip, deflate, br
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: _xsrf=2|752a7959|d4f14de1fdacb143486d639dcc5a5bd3|1581382035; username-localhost-8888="2|1:0|10:1581383620|23:username-localhost-8888|44:Zjk1ZTJiMjRjZDY2NGRjNGE0OWViMzMyYWU5ZDg1NDM=|bb9d5b9d49b537e940d857f73776c8d785f9ed5f09f9cef38cc6b86f55c5f8b9"




In [5]:
import subprocess

output = subprocess.check_output(['python.exe', 'test.py'])
print(type(output))    #bytes array
print(output)
print(type(output.decode()))    #string
print(output.decode())          #bytes array to string. 한글은 안 됨.
#print(output.decode('cp949'))

<class 'bytes'>
b'12\r\na\xb4\xf5\xc7\xcf\xb1\xe2b\r\n'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 5: invalid start byte